Code to request for API: 
https://www.dataquest.io/blog/python-api-tutorial/

Dataset:
https://data.gov.sg/dataset/realtime-weather-readings

In command line: 
    pip install requests

API Request: https://www.dataquest.io/blog/python-api-tutorial/

Dataset: https://data.gov.sg/dataset/realtime-weather-readings

Issue of using weather:
- date they posted the review can be different from the day they visited the attractions --> we can use weekly average?

Note: 
- Starting month of the data is rounded up

In [1]:
import requests
import json
import pandas as pd
import time

## List of dates (From Jan 2017 to Jun 2021)

In [2]:
year = 2017
month = 1
date = 1
day30 = [2, 4, 6, 9, 11] # months with 30 days

date_list = []
while year <= 2021:
    while (year < 2021 and month <= 12) or (year == 2021 and month <= 6):
        if year == 2020 and month == 2: # 2020 is a leap year
            while date <= 29: 
                if date < 10:                
                    date_list.append(f'{year}-0{month}-0{date}')
                else:
                    date_list.append(f'{year}-0{month}-{date}')
                date += 1

        elif year != 2020 and month == 2:
            while date <= 28:
                if date < 10:                
                    date_list.append(f'{year}-0{month}-0{date}')
                else:
                    date_list.append(f'{year}-0{month}-{date}')
                date += 1

        elif month in day30:
            while date <= 30:
                if month < 10: 
                    if date < 10:                
                        date_list.append(f'{year}-0{month}-0{date}')
                    else:
                        date_list.append(f'{year}-0{month}-{date}')
                else:
                    date_list.append(f'{year}-{month}-{date}')
                date += 1

        elif month not in day30 and month != 2:
            while date <= 31:
                if month < 10: 
                    if date < 10:                
                        date_list.append(f'{year}-0{month}-0{date}')
                    else:
                        date_list.append(f'{year}-0{month}-{date}')
                else:
                    date_list.append(f'{year}-{month}-{date}')
                date += 1

        date = 1
        month += 1

    month = 1
    year += 1


date_list[-1] # check for last date

'2021-06-30'

## Air Temperature (From June 2016)

In [3]:
count = 0        
year = 2017

at_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
at_data = []

for date in date_list:
    if count == 7:    # remove this if statement when scrapping all data
        at_readings_df = pd.DataFrame(data = at_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        at_readings_df.to_csv(f'./Data/Weather Raw Data/air_temp_{year}.csv', index=False)     
        break          

    if count == 20: 
        time.sleep(5)
        count = 0

    if int(date[:4]) - 1 == year or date == date_list[-1]:   # creat new csv for every year
        at_readings_df = pd.DataFrame(data = at_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        at_readings_df.to_csv(f'./Data/Weather Raw Data/air_temp_{year}.csv', index=False)     
        at_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings']) 
        year += 1 

    try:
        air_temp = requests.get(f'https://api.data.gov.sg/v1/environment/air-temperature?date={date}')
        print(date)

        at_stations = air_temp.json()["metadata"]["stations"]
        at_data_stations = {}
        for i in at_stations: 
            at_data_stations[i['id']] = {'name':  i['name'], 'latitude': i['location']['latitude'], 'longitude': i['location']['longitude'] }

        at_readings = air_temp.json()["items"]

        for i in at_readings: 
            for r in i['readings']:
                at_data.append([r['station_id'], at_data_stations[r['station_id']]['name'], at_data_stations[r['station_id']]['latitude'], at_data_stations[r['station_id']]['longitude'], i['timestamp'], r['value']])

    except:
        continue

    count += 1


2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07


In [4]:
at_readings_df

,station_id,station_name,latitude,longitude,timestamp,readings
0,S109,Ang Mo Kio Avenue 5,1.37640,103.84920,2017-01-01T00:00:59+08:00,26.0
1,S117,Banyan Road,1.25600,103.67900,2017-01-01T00:00:59+08:00,26.9
2,S107,East Coast Parkway,1.31350,103.96250,2017-01-01T00:00:59+08:00,27.1
3,S43,Kim Chuan Road,1.33990,103.88780,2017-01-01T00:00:59+08:00,26.9
4,S108,Marina Gardens Drive,1.27990,103.87030,2017-01-01T00:00:59+08:00,27.0
...,...,...,...,...,...,...
180387,S24B,Upper Changi Road North,1.36780,103.99800,2017-01-07T23:59:59+08:00,27.8
180388,S24,Upper Changi Road North,1.36780,103.98260,2017-01-07T23:59:59+08:00,27.4
180389,S116,West Coast Highway,1.28100,103.75400,2017-01-07T23:59:59+08:00,29.4
180390,S104,Woodlands Avenue 9,1.44387,103.78538,2017-01-07T23:59:59+08:00,27.8


## Rainfall (From January 2017)

In [5]:
count = 0              
year = 2017

rf_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
rf_data = []

for date in date_list:
    if count == 7:      # remove this if statement when scrapping all data
        rf_readings_df = pd.DataFrame(data = rf_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        rf_readings_df.to_csv(f'./Data/Weather Raw Data/rainfall_{year}.csv', index=False)     
        break          

    if count == 20: 
        time.sleep(5)
        count = 0

    if int(date[:4]) - 1 == year or date == date_list[-1]:   # creat new csv for every year
        rf_readings_df = pd.DataFrame(data = rf_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        rf_readings_df.to_csv(f'./Data/Weather Raw Data/rainfall_{year}.csv', index=False)     
        rf_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings']) 
        year += 1 

    try:
        rainfall = requests.get(f'https://api.data.gov.sg/v1/environment/rainfall?date={date}')
        print(date)

        rf_stations = rainfall.json()["metadata"]["stations"]
        rf_data_stations = {}
        for i in rf_stations: 
            rf_data_stations[i['id']] = {'name':  i['name'], 'latitude': i['location']['latitude'], 'longitude': i['location']['longitude'] }

        rf_readings = rainfall.json()["items"]

        for i in rf_readings: 
            for r in i['readings']:
                rf_data.append([r['station_id'], rf_data_stations[r['station_id']]['name'], rf_data_stations[r['station_id']]['latitude'], rf_data_stations[r['station_id']]['longitude'], i['timestamp'], r['value']])

    except:
        continue

    count += 1

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07


In [6]:
rf_readings_df

,station_id,station_name,latitude,longitude,timestamp,readings
0,S77,Alexandra Road,1.29370,103.81250,2017-01-01T00:04:59+08:00,0.0
1,S109,Ang Mo Kio Avenue 5,1.37640,103.84920,2017-01-01T00:04:59+08:00,0.0
2,S117,Banyan Road,1.25600,103.67900,2017-01-01T00:04:59+08:00,0.0
3,S55,Buangkok Green,1.38370,103.88600,2017-01-01T00:04:59+08:00,0.0
4,S64,Bukit Panjang Road,1.38240,103.76030,2017-01-01T00:04:59+08:00,0.0
...,...,...,...,...,...,...
112108,S08,Upper Thomson Road,1.37010,103.82710,2017-01-07T23:59:59+08:00,0.0
112109,S116,West Coast Highway,1.28100,103.75400,2017-01-07T23:59:59+08:00,0.0
112110,S104,Woodlands Avenue 9,1.44387,103.78538,2017-01-07T23:59:59+08:00,0.0
112111,S100,Woodlands Road,1.41720,103.74855,2017-01-07T23:59:59+08:00,0.0


## Relative Humidity (From June 2016)

In [7]:
count = 0             
year = 2017

rh_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
rh_data = []

for date in date_list:
    if count == 7:    # remove this if statement when scrapping all data
        rh_readings_df = pd.DataFrame(data = rh_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        rh_readings_df.to_csv(f'./Data/Weather Raw Data/relative_humidity_{year}.csv', index=False)     
        break          

    if count == 20: 
        time.sleep(5)
        count = 0

    if int(date[:4]) - 1 == year or date == date_list[-1]:   # creat new csv for every year
        rh_readings_df = pd.DataFrame(data = rh_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        rh_readings_df.to_csv(f'./Data/Weather Raw Data/relative_humidity_{year}.csv', index=False)     
        rh_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings']) 
        year += 1 

    try:
        relative_humidity = requests.get(f'https://api.data.gov.sg/v1/environment/relative-humidity?date={date}')
        print(date)

        rh_stations = relative_humidity.json()["metadata"]["stations"]
        rh_data_stations = {}
        for i in rh_stations: 
            rh_data_stations[i['id']] = {'name':  i['name'], 'latitude': i['location']['latitude'], 'longitude': i['location']['longitude'] }

        rh_readings = relative_humidity.json()["items"]

        for i in rh_readings: 
            for r in i['readings']:
                rh_data.append([r['station_id'], rh_data_stations[r['station_id']]['name'], rh_data_stations[r['station_id']]['latitude'], rh_data_stations[r['station_id']]['longitude'], i['timestamp'], r['value']])

    except:
        continue
    
    count += 1

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07


In [8]:
rh_readings_df

,station_id,station_name,latitude,longitude,timestamp,readings
0,S109,Ang Mo Kio Avenue 5,1.37640,103.84920,2017-01-01T00:00:59+08:00,90.6
1,S117,Banyan Road,1.25600,103.67900,2017-01-01T00:00:59+08:00,82.0
2,S107,East Coast Parkway,1.31350,103.96250,2017-01-01T00:00:59+08:00,89.7
3,S43,Kim Chuan Road,1.33990,103.88780,2017-01-01T00:00:59+08:00,88.1
4,S108,Marina Gardens Drive,1.27990,103.87030,2017-01-01T00:00:59+08:00,87.2
...,...,...,...,...,...,...
148648,S115,Tuas South Avenue 3,1.29377,103.61843,2017-01-07T23:59:59+08:00,78.6
148649,S24,Upper Changi Road North,1.36780,103.98260,2017-01-07T23:59:59+08:00,90.0
148650,S116,West Coast Highway,1.28100,103.75400,2017-01-07T23:59:59+08:00,85.2
148651,S104,Woodlands Avenue 9,1.44387,103.78538,2017-01-07T23:59:59+08:00,98.2


## Wind Direction (From January 2017)

In [9]:
count = 0              
year = 2017

wd_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
wd_data = []

for date in date_list:
    if count == 7:    # remove this if statement when scrapping all data
        wd_readings_df = pd.DataFrame(data = wd_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        wd_readings_df.to_csv(f'./Data/Weather Raw Data/wind_direction_{year}.csv', index=False)     
        break          

    if count == 20: 
        time.sleep(5)
        count = 0

    if int(date[:4]) - 1 == year or date == date_list[-1]:   # creat new csv for every year
        wd_readings_df = pd.DataFrame(data = wd_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        wd_readings_df.to_csv(f'./Data/Weather Raw Data/wind_direction_{year}.csv', index=False)     
        wd_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings']) 
        year += 1 

    try:
        wind_direc = requests.get(f'https://api.data.gov.sg/v1/environment/wind-direction?date={date}')
        print(date)

        wd_stations = wind_direc.json()["metadata"]["stations"]
        wd_data_stations = {}
        for i in wd_stations: 
            wd_data_stations[i['id']] = {'name':  i['name'], 'latitude': i['location']['latitude'], 'longitude': i['location']['longitude'] }

        wd_readings = wind_direc.json()["items"]

        for i in wd_readings: 
            for r in i['readings']:
                wd_data.append([r['station_id'], wd_data_stations[r['station_id']]['name'], wd_data_stations[r['station_id']]['latitude'], wd_data_stations[r['station_id']]['longitude'], i['timestamp'], r['value']])

    except:
        continue

    count += 1

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07


In [10]:
wd_readings_df

,station_id,station_name,latitude,longitude,timestamp,readings
0,S109,Ang Mo Kio Avenue 5,1.37640,103.84920,2017-01-01T00:00:59+08:00,25
1,S117,Banyan Road,1.25600,103.67900,2017-01-01T00:00:59+08:00,345
2,S50,Clementi Road,1.33370,103.77680,2017-01-01T00:00:59+08:00,337
3,S107,East Coast Parkway,1.31350,103.96250,2017-01-01T00:00:59+08:00,1
4,S43,Kim Chuan Road,1.33990,103.88780,2017-01-01T00:00:59+08:00,38
...,...,...,...,...,...,...
189027,S24B,Upper Changi Road North,1.36780,103.99800,2017-01-07T23:59:59+08:00,78
189028,S24,Upper Changi Road North,1.36780,103.98260,2017-01-07T23:59:59+08:00,85
189029,S116,West Coast Highway,1.28100,103.75400,2017-01-07T23:59:59+08:00,51
189030,S104,Woodlands Avenue 9,1.44387,103.78538,2017-01-07T23:59:59+08:00,46


## Wind Speed (From January 2017)

In [11]:
count = 0              
year = 2017

ws_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
ws_data = []

for date in date_list:
    if count == 7:    # remove this if statement when scrapping all data
        ws_readings_df = pd.DataFrame(data = ws_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        ws_readings_df.to_csv(f'./Data/Weather Raw Data/wind_speed_{year}.csv', index=False)     
        break          

    if count == 20: 
        time.sleep(5)
        count = 0

    if int(date[:4]) - 1 == year or date == date_list[-1]:   # creat new csv for every year
        ws_readings_df = pd.DataFrame(data = ws_data, columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings'])
        ws_readings_df.to_csv(f'./Data/Weather Raw Data/wind_speed_{year}.csv', index=False)     
        ws_readings_df = pd.DataFrame(columns = ['station_id', 'station_name', 'latitude', 'longitude', 'timestamp', 'readings']) 
        year += 1 

    try:
        wind_speed = requests.get(f'https://api.data.gov.sg/v1/environment/wind-speed?date={date}')
        print(date)

        ws_stations = wind_speed.json()["metadata"]["stations"]
        ws_data_stations = {}
        for i in ws_stations: 
            ws_data_stations[i['id']] = {'name':  i['name'], 'latitude': i['location']['latitude'], 'longitude': i['location']['longitude'] }

        ws_readings = wind_speed.json()["items"]

        for i in ws_readings: 
            for r in i['readings']:
                ws_data.append([r['station_id'], ws_data_stations[r['station_id']]['name'], ws_data_stations[r['station_id']]['latitude'], ws_data_stations[r['station_id']]['longitude'], i['timestamp'], r['value']])

    except:
        continue

    count += 1


2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07


In [12]:
ws_readings_df

,station_id,station_name,latitude,longitude,timestamp,readings
0,S109,Ang Mo Kio Avenue 5,1.37640,103.84920,2017-01-01T00:00:59+08:00,2.0
1,S117,Banyan Road,1.25600,103.67900,2017-01-01T00:00:59+08:00,2.2
2,S50,Clementi Road,1.33370,103.77680,2017-01-01T00:00:59+08:00,5.7
3,S107,East Coast Parkway,1.31350,103.96250,2017-01-01T00:00:59+08:00,1.4
4,S43,Kim Chuan Road,1.33990,103.88780,2017-01-01T00:00:59+08:00,2.2
...,...,...,...,...,...,...
190446,S24B,Upper Changi Road North,1.36780,103.99800,2017-01-07T23:59:59+08:00,4.0
190447,S24,Upper Changi Road North,1.36780,103.98260,2017-01-07T23:59:59+08:00,4.1
190448,S116,West Coast Highway,1.28100,103.75400,2017-01-07T23:59:59+08:00,2.3
190449,S104,Woodlands Avenue 9,1.44387,103.78538,2017-01-07T23:59:59+08:00,5.3
